In [1]:
import pandas as pd
import random
import sys
import numpy as np

sys.path.append("../scripts")
from scripts.prompt_gen import generate_prompt
from scripts.translate_summary_json import translate_summary

%load_ext autoreload
%autoreload 2

In [22]:
df = pd.read_csv("../outputs/cleaned_augmented_clinical_notes.csv")
q_df = pd.read_csv("../outputs/questions.csv")

In [274]:
def generate_record_dict ():
    record_dict = {
        "idx" : 0,
        "full_note" : "",
        "summary" : "",
        "questions" : "",
        "section_1" : "",
        "section_2" : "",
        "section_1_ground_truth_summary" : "",
        "section_2_ground_truth_summary" : "",
        "ground_truth_summary" : "",
        "prompt" : "",
        "ground_truth_summary_sections" : "",
    }
    return record_dict
    

record_list = []
for i in range(100):
    prompt_details = generate_prompt(df, q_df, return_prompt_string= False)
    
   
    record_dict = generate_record_dict()
    record_dict["idx"] = prompt_details["idx"]
    record_dict["prompt"] = prompt_details["prompt"]
    record_dict["full_note"] = prompt_details["full_note"].iloc[0]
    record_dict["summary"] = prompt_details["summary"]
    record_dict["questions"] = prompt_details["questions"]
    record_dict["section_1"] = prompt_details["sections"][0]
    record_dict["section_2"] = prompt_details["sections"][1]
    record_dict["ground_truth_summary"] = prompt_details["translated_summary"]

    record_list.append(record_dict)

In [275]:
for record_dict in record_list:
    translated_summary, summary_dict = translate_summary(record_dict["summary"])
    record_dict["ground_truth_summary"] = translated_summary
    record_dict["section_1_ground_truth_summary"] = "".join(summary_dict[record_dict["section_1"]])
    record_dict["section_2_ground_truth_summary"] = "".join(summary_dict[record_dict["section_2"]])
    record_dict["ground_truth_summary_sections"] = record_dict["section_1_ground_truth_summary"] + " \n" + record_dict["section_2_ground_truth_summary"]

In [276]:
df_records = pd.DataFrame(record_list)
(df_records["section_2"].value_counts() + df_records["section_1"].value_counts()).sort_values(ascending = False)

Symptoms                   23
Medical Examinations       22
Treatments                 21
Admission                  20
Diagnosis Tests            20
Discharge                  20
Patient Medical History    20
Patient Information        19
Visit Motivation           19
Surgeries                  16
Name: count, dtype: int64

In [277]:
df_records

,idx,full_note,summary,questions,section_1,section_2,section_1_ground_truth_summary,section_2_ground_truth_summary,ground_truth_summary,prompt,ground_truth_summary_sections
0,128470,Our patient is a 42-year-old male who initiall...,{'visit motivation': 'Intermittent right knee ...,[[What is the motivation for the patient's vis...,Visit Motivation,Patient Medical History,\nVisit Motivation:The patient reported Interm...,\nMedical History:The patient's physiological ...,Visit Motivation: The patient reported Intermi...,[Start Full note:]\nOur patient is a 42-year-o...,\nVisit Motivation:The patient reported Interm...
1,118221,A 67-year-old man with a 40 pack-year smoking ...,{'visit motivation': 'Initially presented with...,[[Are there any specific details mentioned abo...,Surgeries,Patient Information,\nSurgeries:A AV nodal ablation surgery was pe...,\nPatient Information:The patient is 67 and Male.,Visit Motivation: The patient reported Initial...,[Start Full note:]\nA 67-year-old man with a 4...,\nSurgeries:A AV nodal ablation surgery was pe...
2,92394,A 13-year-old Caucasian boy presented with acu...,{'visit motivation': 'Intermittent abdominal p...,[[What medical condition was the test intended...,Diagnosis Tests,Treatments,\nDiagnosis Tests:The CT scan of the abdomen a...,Error in treatments information: 'NoneType' ob...,Visit Motivation: The patient reported Intermi...,[Start Full note:]\nA 13-year-old Caucasian bo...,\nDiagnosis Tests:The CT scan of the abdomen a...
3,163777,A 51-years old man presented with a clinical d...,{'visit motivation': 'Clinical diagnosis of mu...,"[[What diagnostic test(s) were performed?, Wha...",Diagnosis Tests,Patient Information,\nDiagnosis Tests:The Intraoperative histologi...,\nPatient Information:The patient is 51-years ...,Visit Motivation: The patient reported Clinica...,[Start Full note:]\nA 51-years old man present...,\nDiagnosis Tests:The Intraoperative histologi...
4,132520,A 68-year-old right-handed man with an educati...,{'visit motivation': 'Weakness in the bilatera...,[[Are there any additional details mentioned a...,Treatments,Surgeries,\nTreatments:The patient received Steroid trea...,\nSurgeries: No information,Visit Motivation: The patient reported Weaknes...,[Start Full note:]\nA 68-year-old right-handed...,\nTreatments:The patient received Steroid trea...
...,...,...,...,...,...,...,...,...,...,...,...
95,7199,A 45-year-old man was admitted for a progressi...,{'visit motivation': 'Progressively worsening ...,[[What is the motivation for the patient's vis...,Visit Motivation,Surgeries,\nVisit Motivation:The patient reported Progre...,\nSurgeries:A Digital subtraction angiography ...,Visit Motivation: The patient reported Progres...,[Start Full note:]\nA 45-year-old man was admi...,\nVisit Motivation:The patient reported Progre...
96,25724,"A 26-year-old woman (gravida 1, para 0) at 24 ...",{'visit motivation': 'Intrauterine fetal death...,[[What is the motivation for the patient's vis...,Visit Motivation,Patient Medical History,\nVisit Motivation:The patient reported Intrau...,\nMedical History:The patient's physiological ...,Visit Motivation: The patient reported Intraut...,[Start Full note:]\nA 26-year-old woman (gravi...,\nVisit Motivation:The patient reported Intrau...
97,167604,"A 38-year-old North African man, with no past ...",{'visit motivation': 'painless left scrotal ma...,[[What is the motivation for the patient's vis...,Visit Motivation,Patient Information,\nVisit Motivation:The patient reported painle...,\nPatient Information:The patient is 38 and Ma...,Visit Motivation: The patient reported painles...,[Start Full note:]\nA 38-year-old North Africa...,\nVisit Motivation:The patient reported painle...
98,101821,Mrs. S is a 56-year-old lady who was brought t...,{'visit motivation': 'History of alleged snake...,[[What medication or treatment was administere...,Treatments,Medical Examinations,\nTreatments:The patient received Mechanical v...,\nMedical Examinations

In [278]:
df_records.to_csv("../outputs/eval.csv", index=False)

In [2]:
preds = pd.read_csv("../outputs/preds.csv")

In [7]:
preds["ground_truth_summary_sections"] = preds["section_1_ground_truth_summary"] + " \n" + preds["section_2_ground_truth_summary"]

In [8]:
preds.to_csv("../outputs/preds.csv", index=False)

In [9]:
eval_df = pd.read_csv("../outputs/eval.csv")

In [10]:
eval_df["ground_truth_summary_section"] = eval_df["section_1_ground_truth_summary"] + " \n" + eval_df["section_2_ground_truth_summary"]

In [11]:
eval_df

,idx,full_note,summary,questions,section_1,section_2,section_1_ground_truth_summary,section_2_ground_truth_summary,ground_truth_summary,prompt,ground_truth_summary_sections,ground_truth_summary_section
0,128470,Our patient is a 42-year-old male who initiall...,{'visit motivation': 'Intermittent right knee ...,"[[""What is the motivation for the patient's vi...",Visit Motivation,Patient Medical History,\nVisit Motivation:The patient reported Interm...,\nMedical History:The patient's physiological ...,Visit Motivation: The patient reported Intermi...,[Start Full note:]\nOur patient is a 42-year-o...,\nVisit Motivation:The patient reported Interm...,\nVisit Motivation:The patient reported Interm...
1,118221,A 67-year-old man with a 40 pack-year smoking ...,{'visit motivation': 'Initially presented with...,[['Are there any specific details mentioned ab...,Surgeries,Patient Information,\nSurgeries:A AV nodal ablation surgery was pe...,\nPatient Information:The patient is 67 and Male.,Visit Motivation: The patient reported Initial...,[Start Full note:]\nA 67-year-old man with a 4...,\nSurgeries:A AV nodal ablation surgery was pe...,\nSurgeries:A AV nodal ablation surgery was pe...
2,92394,A 13-year-old Caucasian boy presented with acu...,{'visit motivation': 'Intermittent abdominal p...,[['What medical condition was the test intende...,Diagnosis Tests,Treatments,\nDiagnosis Tests:The CT scan of the abdomen a...,Error in treatments information: 'NoneType' ob...,Visit Motivation: The patient reported Intermi...,[Start Full note:]\nA 13-year-old Caucasian bo...,\nDiagnosis Tests:The CT scan of the abdomen a...,\nDiagnosis Tests:The CT scan of the abdomen a...
3,163777,A 51-years old man presented with a clinical d...,{'visit motivation': 'Clinical diagnosis of mu...,"[['What diagnostic test(s) were performed?', '...",Diagnosis Tests,Patient Information,\nDiagnosis Tests:The Intraoperative histologi...,\nPatient Information:The patient is 51-years ...,Visit Motivation: The patient reported Clinica...,[Start Full note:]\nA 51-years old man present...,\nDiagnosis Tests:The Intraoperative histologi...,\nDiagnosis Tests:The Intraoperative histologi...
4,132520,A 68-year-old right-handed man with an educati...,{'visit motivation': 'Weakness in the bilatera...,[['Are there any additional details mentioned ...,Treatments,Surgeries,\nTreatments:The patient received Steroid trea...,\nSurgeries: No information,Visit Motivation: The patient reported Weaknes...,[Start Full note:]\nA 68-year-old right-handed...,\nTreatments:The patient received Steroid trea...,\nTreatments:The patient received Steroid trea...
...,...,...,...,...,...,...,...,...,...,...,...,...
95,7199,A 45-year-old man was admitted for a progressi...,{'visit motivation': 'Progressively worsening ...,"[[""What is the motivation for the patient's vi...",Visit Motivation,Surgeries,\nVisit Motivation:The patient reported Progre...,\nSurgeries:A Digital subtraction angiography ...,Visit Motivation: The patient reported Progres...,[Start Full note:]\nA 45-year-old man was admi...,\nVisit Motivation:The patient reported Progre...,\nVisit Motivation:The patient reported Progre...
96,25724,"A 26-year-old woman (gravida 1, para 0) at 24 ...",{'visit motivation': 'Intrauterine fetal death...,"[[""What is the motivation for the patient's vi...",Visit Motivation,Patient Medical History,\nVisit Motivation:The patient reported Intrau...,\nMedical History:The patient's physiological ...,Visit Motivation: The patient reported Intraut...,[Start Full note:]\nA 26-year-old woman (gravi...,\nVisit Motivation:The patient reported Intrau...,\nVisit Motivation:The patient reported Intrau...
97,167604,"A 38-year-old North African man, with no past ...",{'visit motivation': 'painless left scrotal ma...,"[[""What is the motivation for the patient's vi...",Visit Motivation,Patient Information,\nVisit Motivation:The patient reported painle...,\nPatient Information:The patient is 38 and Ma...,Visit Motivation: T

In [12]:
eval_df.to_csv("../outputs/eval.csv", index=False)